In [7]:
import pandas as pd
from bertopic import BERTopic
import pandas as pd
import spacy
import nltk
import re
import string
import numpy as np
from nltk.corpus import stopwords
#import gensim
#from gensim import corpora
#import matplotlib.pyplot as plt
#import stanza
import spacy
from tqdm import tqdm

In [90]:
from bertopic.backend import BaseEmbedder
from bertopic.cluster import BaseCluster
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.dimensionality import BaseDimensionalityReduction

In [3]:
data = pd.read_csv('cleaned_mbs_total.csv')
data.head()

,Unnamed: 0,traveller_username,review_title,review_text,travel_type,traveller_country_origin,traveller_total_contributions,traveller_total_helpful_contributions,rating,valid_rating,label,cleaned_review,combined_review,date,covid,year,stem_review,lem_review
0,0,Erica G,"Sick in Singapore, and MBS staff were amazing!","I was in Singapore on business and, unfortunat...",Trip type: Travelled on business,"Arlington Heights, Illinois",105.0,62.0,5.0,True,Positive,sick singapore mbs staff amazing singapore bus...,"Sick in Singapore, and MBS staff were amazing!...",2023-08-01,PostCovid,2023,sick singapor mb staff amaz singapor busi unfo...,sick singapore mbs staff amazing singapore bus...
1,1,HJay,Lovely place to go whatever you plan to do!,Whether it’s to soak up the Marina Bay citysca...,NaN,"Perth, Australia",14.0,11.0,NaN,False,NaN,lovely place go whatever plan whether soak mar...,Lovely place to go whatever you plan to do! Wh...,2023-04-01,PostCovid,2023,love place go whatev plan whether soak marina ...,lovely place go whatever plan whether soak mar...
2,2,TaM,Thank you for the unforgettable memories,I stayed at Marina Bay Sands to propose to my ...,Trip type: Travelled as a couple,NaN,1.0,NaN,5.0,True,Positive,thank unforgettable memories stayed marina bay...,Thank you for the unforgettable memories I sta...,2023-09-01,PostCovid,2023,thank unforgett memori stay marina bay sand pr...,thank unforgettable memory stay marina bay san...
3,3,Praxmeyer,Amazing hotel but not sure I’d do it again,We stayed one night in the hotel. The good par...,Trip type: Travelled as a couple,"Napier, New Zealand",444.0,232.0,4.0,True,Positive,amazing hotel sure stayed one night hotel good...,Amazing hotel but not sure I’d do it again We ...,2023-09-01,PostCovid,2023,amaz hotel sure stay one night hotel good part...,amazing hotel sure stay one night hotel good p...
4,4,TravelWriter74,Stunning hotel and overall an amazing experience,We have just returned from a super couple of d...,Trip type: Travelled with family,"London, United Kingdom",166.0,114.0,5.0,True,Positive,stunning hotel overall amazing experience retu...,Stunning hotel and overall an amazing experien...,2023-08-01,PostCovid,2023,stun hotel overal amaz experi return super cou...,stunning hotel overall amazing experience retu...


In [57]:
nlp = spacy.load("en_core_web_sm")
data['aspect_sentiment'] = np.nan

In [87]:
for i in range(len(data)):
    sentence = data['combined_review'][i]
    doc = nlp(sentence)
    aspect_sentiment = []
    for word in doc:
        if word.pos_ == 'NOUN': 
            #print(word)
            for j in word.lefts:
                #print(j, word, j.dep_, j.pos_)
                if j.dep_ == 'amod' and j.pos_ == 'ADJ':
                    tup = (word, j)
                    #print(tup)
                    aspect_sentiment.append(tup)
                for k in j.lefts:
                    if k.dep_ == 'advmod':
                        #print(word, j ,k)
                        tup = (word, k.text + ' ' + j.text)
                        aspect_sentiment.append(tup)
                        if (word, j) in aspect_sentiment:
                            aspect_sentiment.remove((word, j))
    """
        if word.pos_ == 'VERB':
            for j in word.lefts:
                if j.dep_ == 'advmod' and j.pos_ == 'ADV':
                    tup = (word, j)
                    aspect_sentiment.append(tup)
                if j.dep_ == 'neg' and j.pos_ == 'ADV':
                    tup = (word, j)
                    aspect_sentiment.append(tup)
            for j in word.rights:
                if j.dep_ == 'advmod'and j.pos_ == 'ADV':
                    tup = (word, j)
                    aspect_sentiment.append(tup)
    aspect_sentiment
    """
    data['aspect_sentiment'][i] = aspect_sentiment

/var/folders/t_/v1j1crh97wl_c7mn9f9gq49c0000gn/T/ipykernel_39399/390396905.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['aspect_sentiment'][i] = aspect_sentiment
/var/folders/t_/v1j1crh97wl_c7mn9f9gq49c0000gn/T/ipykernel_39399/390396905.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[(days, several), (bed, 'verycomfy'), (circumstance, unfortunate), (job, great)]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data['aspect_sentiment'][i] = aspect_sentiment


In [88]:
data['aspect_sentiment']

0        [(days, several), (bed, verycomfy), (circumsta...
1        [(place, Lovely), (items, mostexpensive), (chi...
2        [(memories, unforgettable), (weeks, about3), (...
3        [(hotel, Amazing), (parts, good), (view, amazi...
4        [(hotel, Stunning), (experience, amazing), (co...
                               ...                        
10518    [(place, Best), (year, new), (week, last), (ye...
10519    [(view, best), (months, about12), (things, mos...
10520    [(food, better), (food, same), (experience, su...
10521    [(hotels, different), (pool, famous), (shops, ...
10522                    [(scenery, clean), (area, other)]
Name: aspect_sentiment, Length: 10523, dtype: object